In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("Libraries loaded successfully.")
print("Using statsmodels for GLM (sklearn doesn't support all GLM families)")

## The GLM Family Tree

| Distribution | Response Type | Link Function | Use Case |
|-------------|---------------|---------------|----------|
| Gaussian | Continuous, unbounded | Identity | Standard regression |
| Binomial | Binary (0/1) | Logit | Classification |
| Poisson | Counts (0, 1, 2, ...) | Log | Event counts |
| **Gamma** | Positive continuous | Log or Inverse | Costs, times, claims |
| **Negative Binomial** | Overdispersed counts | Log | Counts with extra variance |

---

## Part 1: Gamma Regression

Gamma regression is perfect for modeling **positive continuous** data that is often **right-skewed**. Common applications include:
- Insurance claim amounts (always positive, skewed right)
- Hospital costs (always positive, varies widely)
- Time to failure (always positive)

### Why not just use linear regression with log-transformed response?

You could, but Gamma GLM has advantages:
1. Models the original scale directly (no back-transformation bias)
2. Handles heteroscedasticity naturally (variance increases with mean)
3. Proper inference on the original scale

In [ ]:
# Simulate insurance claims data
n = 500

# Features
age = np.random.uniform(20, 70, n)
risk_score = np.random.uniform(0, 100, n)
years_customer = np.random.uniform(0, 20, n)

# True relationship (log link): log(mu) = b0 + b1*age + b2*risk + b3*years
log_mu = 5 + 0.02 * age + 0.03 * risk_score - 0.05 * years_customer
mu = np.exp(log_mu)

# Generate Gamma-distributed claims (shape=2 gives moderate skew)
shape = 2
scale = mu / shape
claims = np.random.gamma(shape, scale)

# Create dataframe
data = pd.DataFrame({
    'age': age,
    'risk_score': risk_score,
    'years_customer': years_customer,
    'claim_amount': claims
})

print("Simulated Insurance Claims Data:")
print(f"  Samples: {n}")
print(f"  Claim amount range: ${claims.min():.0f} to ${claims.max():.0f}")
print(f"  Mean claim: ${claims.mean():.0f}")
print(f"  Median claim: ${np.median(claims):.0f} (< mean = right-skewed)")
print(data.head())

In [ ]:
# Visualize the distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram of claims
axes[0].hist(claims, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(claims.mean(), color='red', linestyle='--', label=f'Mean: ${claims.mean():.0f}')
axes[0].axvline(np.median(claims), color='green', linestyle='--', label=f'Median: ${np.median(claims):.0f}')
axes[0].set_xlabel('Claim Amount ($)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Insurance Claims\n(Gamma-distributed, right-skewed)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Q-Q plot against Gamma
stats.probplot(claims, dist=stats.gamma, sparams=(shape,), plot=axes[1])
axes[1].set_title('Q-Q Plot: Claims vs Gamma Distribution', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("OBSERVATION:")
print("  - Claims are strictly positive (no negative values possible)")
print("  - Distribution is right-skewed (mean > median)")
print("  - Linear regression would be inappropriate (assumes normal errors)")

In [ ]:
# Fit Gamma GLM
X = data[['age', 'risk_score', 'years_customer']]
X = sm.add_constant(X)
y = data['claim_amount']

# Gamma with log link
gamma_model = sm.GLM(y, X, family=sm.families.Gamma(link=sm.families.links.Log()))
gamma_results = gamma_model.fit()

print("=" * 60)
print("GAMMA REGRESSION RESULTS")
print("=" * 60)
print(gamma_results.summary())

In [ ]:
print("\nINTERPRETATION (Log Link):")
print("-" * 60)

for name, coef in zip(['const', 'age', 'risk_score', 'years_customer'], gamma_results.params):
    if name == 'const':
        print(f"Intercept: {coef:.4f}")
        print(f"  → Baseline claim when all features = 0: ${np.exp(coef):.0f}")
    else:
        pct_change = (np.exp(coef) - 1) * 100
        print(f"{name}: {coef:.4f}")
        print(f"  → Each unit increase multiplies claim by {np.exp(coef):.3f}")
        print(f"  → Equivalent to {pct_change:+.1f}% change per unit")
    print()

print("PRACTICAL MEANING:")
print("  - Older customers: Higher claims (age coefficient positive)")
print("  - Higher risk scores: Higher claims (risk coefficient positive)")
print("  - Longer customer tenure: Lower claims (loyalty discount effect)")

## Part 2: Negative Binomial Regression

Poisson regression assumes **mean = variance**. But real count data often has **overdispersion** (variance > mean). Negative Binomial adds an extra parameter to handle this.

### When to suspect overdispersion?
- Variance of counts is much larger than mean
- Too many zeros in the data
- Individual heterogeneity (some subjects inherently have more events)

In [ ]:
# Simulate hospital visits data (overdispersed counts)
n = 500

# Features
age = np.random.uniform(20, 80, n)
chronic_conditions = np.random.poisson(1.5, n)
income_level = np.random.uniform(1, 5, n)  # 1=low, 5=high

# True relationship (log link)
log_mu = 0.5 + 0.02 * age + 0.4 * chronic_conditions - 0.2 * income_level
mu = np.exp(log_mu)

# Generate overdispersed counts using Negative Binomial
# NB: can be parameterized as Poisson-Gamma mixture
alpha = 0.5  # dispersion parameter (higher = more overdispersion)
visits = np.random.negative_binomial(1/alpha, 1/(1 + alpha*mu))

data_nb = pd.DataFrame({
    'age': age,
    'chronic_conditions': chronic_conditions,
    'income_level': income_level,
    'hospital_visits': visits
})

print("Simulated Hospital Visits Data:")
print(f"  Mean visits: {visits.mean():.2f}")
print(f"  Variance of visits: {visits.var():.2f}")
print(f"  Variance/Mean ratio: {visits.var()/visits.mean():.2f} (>1 = overdispersed!)")
print(f"\nIf Poisson were correct, variance/mean should be ~1.0")
print(data_nb.head())

In [ ]:
# Compare Poisson vs Negative Binomial
X_nb = data_nb[['age', 'chronic_conditions', 'income_level']]
X_nb = sm.add_constant(X_nb)
y_nb = data_nb['hospital_visits']

# Poisson
poisson_model = sm.GLM(y_nb, X_nb, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Negative Binomial
nb_model = sm.GLM(y_nb, X_nb, family=sm.families.NegativeBinomial())
nb_results = nb_model.fit()

print("=" * 60)
print("POISSON vs NEGATIVE BINOMIAL COMPARISON")
print("=" * 60)

print(f"\n{'Metric':<25} {'Poisson':<15} {'Neg Binomial'}")
print("-" * 60)
print(f"{'AIC':<25} {poisson_results.aic:<15.2f} {nb_results.aic:.2f}")
print(f"{'Deviance':<25} {poisson_results.deviance:<15.2f} {nb_results.deviance:.2f}")
print(f"{'Pearson chi2/df':<25} {poisson_results.pearson_chi2/poisson_results.df_resid:<15.2f} {nb_results.pearson_chi2/nb_results.df_resid:.2f}")

print("\nINTERPRETATION:")
print(f"  - Lower AIC is better: {'Negative Binomial' if nb_results.aic < poisson_results.aic else 'Poisson'} wins")
print(f"  - Pearson chi2/df should be ~1 for good fit")
print(f"    Poisson: {poisson_results.pearson_chi2/poisson_results.df_resid:.2f} (>>1 = overdispersion not captured)")
print(f"    NegBin:  {nb_results.pearson_chi2/nb_results.df_resid:.2f} (closer to 1 = better fit)")

In [ ]:
# Coefficient comparison
print("\n" + "=" * 60)
print("COEFFICIENT COMPARISON")
print("=" * 60)
print(f"\n{'Feature':<25} {'Poisson':<12} {'Poisson SE':<12} {'NegBin':<12} {'NegBin SE'}")
print("-" * 70)

for i, name in enumerate(['const', 'age', 'chronic_conditions', 'income_level']):
    print(f"{name:<25} {poisson_results.params[i]:<12.4f} {poisson_results.bse[i]:<12.4f} {nb_results.params[i]:<12.4f} {nb_results.bse[i]:.4f}")

print("\nKEY INSIGHT:")
print("  - Coefficient estimates are similar between models")
print("  - BUT Poisson standard errors are too small (overconfident!)")
print("  - Negative Binomial gives more honest (larger) standard errors")
print("  - Using Poisson on overdispersed data: Type I error inflation")

## GLM Model Selection Flowchart

```
What type of response variable?
│
├─► Continuous, unbounded → Gaussian (Linear Regression)
│
├─► Binary (0/1) → Binomial (Logistic Regression)
│
├─► Counts (0, 1, 2, ...) → Check for overdispersion
│   ├─► Variance ≈ Mean → Poisson
│   └─► Variance >> Mean → Negative Binomial
│
└─► Positive continuous → Gamma
    (costs, times, claims)
```

---

## Summary

**Gamma Regression:**
- For positive continuous data (costs, times, amounts)
- Handles right-skewed distributions naturally
- Variance proportional to mean squared

**Negative Binomial:**
- For overdispersed count data (variance > mean)
- Adds dispersion parameter to Poisson
- Gives more honest standard errors

**General Advice:**
- Always check residuals and goodness-of-fit
- Compare AIC/BIC across competing models
- Domain knowledge matters - choose distribution that makes physical sense